# Topic Modeling demo (Arabic)

In [5]:
import numpy as np
import numpy.matlib
import pandas as pd
import lda
from glob import glob
from stop_words import get_stop_words

## Loading data
The data we're using for this demo comes from the لنتفق الآن ("Let's Agree Now!") Facebook page. Most posts are in Arabic, and they will address a variety of different topics. Some posts come from the account owner, while the majority will be contributed by visitors to their page. The first five entries are shown below.

In [15]:
xls = pd.concat([pd.read_excel(x) for x in glob('Lets Agree Now*.xls')])
pd.set_option('display.max_colwidth', 1000)
xls[['Author','Contents']].head(5)

,Author,Contents
0,Franc Slame,انتو تفرحو بناقلت نفط ونحن نفرح لتحري أرض الوطن ياانجاس
1,Salma Abu,وأخيرا
2,الحفره الحفره,سبب الخراب والدمار فبراير اليوم الاسود 2011/2/17
3,غدا اجمل,يا خوي اتق الله كيف تحلف بالله علي شي في علم الغيب .. الله وحده العالم بشن حيصير في البلاد .. ومن اللي حيرضى انه بلادنا تنباع ؟؟؟؟
4,Abdo Altwel,مفروض كل واحد فيهم ايروح على رجليه لن يوصل حوشه تو بعدين يتعلمو التشحيط


## Stop words
In the field of Natural Language Processing (NLP), certain words are known as *stop words*. These are very common words -- English examples could include "the", "of", "is", "and", "or", etc. -- that often don't tell us very much about the subject of the sentence that contains them. We'll be using a set of common Arabic stop words that are commonly used by NLP researchers. Here are the first 20, to give a sense of what kind of words we're talking about.

In [7]:
print(get_stop_words('arabic')[:20])

['فى', 'في', 'كل', 'لم', 'لن', 'له', 'من', 'هو', 'هي', 'قوة', 'كما', 'لها', 'منذ', 'وقد', 'ولا', 'نفسه', 'لقاء', 'مقابل', 'هناك', 'وقال']


## Preparing for topic modeling
Topic modeling algorithms often struggle to identify the topic of very short pieces of text. We'll get around this by only paying attention to posts with more than 250 characters -- there are 2,342 such posts. We're also going to simplify things by removing numbers and URLs, since they're not likely to tell us much about topics.

Those long posts are then passed to an algorithm called a *vectorizer* that turns the set of posts into a matrix of numbers, because computers generally prefer to work with numbers. This is done by identifying the *vocabulary* of all words that appear at least once (45,086 of them) and counting the number of times that each word appears in each post. This means we have a 2342x45086 array of numbers (known as a *document-term matrix* or DTM), for 2,342 documents and 45,086 words.

In [26]:
xls['Filtered'] = xls['Contents'].replace(r'\d+','',regex=True)
xls['Filtered'] = xls['Filtered'].replace(r'https?://[\w./]+','',regex=True)

In [27]:
def getlen(x):
    try:
        return(len(x))
    except:
        return(0)
xls['strlen'] = xls['Filtered'].apply(getlen)
long_posts = xls[xls.strlen > 250].reset_index(drop=True)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=get_stop_words('arabic')) 
vect.fit(long_posts.Filtered)
long_dtm = vect.transform(long_posts.Filtered)
vocab = vect.get_feature_names()
long_dtm.shape

(2342, 45086)

## Fitting the topic model
The algorithm we're using is called LDA (Latent Dirichlet Allocation). The basic idea is that we give it a set of documents (or more precisely a DTM), and ask it to identify a specific number of topics (20 in this case). Each document is given a *probability* of belonging to each topic. For each topic, every word has a *weight* -- the words with the highest weights are the ones that are most important to the topic.

Part of the model-fitting algorithm requires a random-number generator -- this means that the results will be slightly different each time. We're going to fit the model twice, because we suspect that topics that are consistent between the two versions of the fitted model will be more reliable.

In [30]:
myLDA1 = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
myLDA1.fit(long_dtm)
myLDA2 = lda.LDA(n_topics=20, n_iter=1500, random_state=2)
myLDA2.fit(long_dtm)

In [31]:
tw1 = myLDA1.topic_word_
tw2 = myLDA2.topic_word_
ldamat1 = myLDA1.transform(long_dtm)
ldamat2 = myLDA2.transform(long_dtm)

## Summarizing topics
We can quickly get a sense of what some of the major topics on the "Let's Agree Now!" Facebook page are by looking at the 8 most heavily-weighted terms in each topic. Here are the results for the first version of the fitted model:

In [32]:
n_top_words = 8

def top_words(topic_dist,n_top_words=10):
    return(np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1])

for i, topic_dist in enumerate(tw1):
    topic_words = top_words(topic_dist,8)
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: الله لله الملك مافي مابعده ليبيا type timeline
Topic 1: أن أو لو ليس جدا al mostakbal الحياة
Topic 2: ليبيا أن إلى المتحدة الليبي حكومة كوبلر السياسي
Topic 3: اللي مش علي والله الي بس البلاد يا
Topic 4: أن إلى وكالة التضامن أنباء أي الليبية أنه
Topic 5: الآن لنتفق لنتفق_الآن type photos timeline ليبيا أو
Topic 6: visa the ﻣﻦ ﺑﺎﻟﻠﻪ on ﺍﻟﻠﻪ ﺇﻻ arrival
Topic 7: اللهم يا الله يارب أن لي خير ربي
Topic 8: وزارة أن الحكومة حكومة وزير الخارجية تشكيلة بأن
Topic 9: الوطني المؤتمر الوفاق حكومة إن السياسي الليبي مجلس
Topic 10: ليبيا الوطن الشعب الله ولكن الليبيين الليبي البلاد
Topic 11: إلى أن داعش ﺍﻛﺒﺮ ﺍﻟﻠﻪ الدولة ليبيا الليبي
Topic 12: النواب مجلس أن المجلس الوفاق الثقة الاتفاق جلسة
Topic 13: الله ال وا محمد ين يا وسلم الل
Topic 14: ليبيا علي الي الجيش الغرب حفتر طرابلس الناس
Topic 15: ليبيا لنتفق المركزي لنتفق_الآن النفط الآن مصرف المواطن
Topic 16: المجلس الرئاسي الوفاق الوطني لحكومة رئيس السراج طرابلس
Topic 17: الآن لنتفق لنتفق_الآن type طرابلس مدينة تشجيع photos
Topic 18: الله الوكيل

To get a sense of how much we can trust the topics, we can look at the same output for the other version of the model and see which topics look similar.

In [33]:
for i, topic_dist in enumerate(tw2):
    topic_words = top_words(topic_dist,8)
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: لنتفق الآن لنتفق_الآن type photos ليبيا timeline أو
Topic 1: النواب مجلس أن جلسة الثقة الوفاق المجلس عضو
Topic 2: الوطني الوفاق حكومة إن السيد الليبي رئيس مجلس
Topic 3: وزارة محمد عل وزيرا سيدنا زليتن آل وعلى
Topic 4: الله الوكيل ونعم حسبي حسبنا شاء فيكم سبحان
Topic 5: المجلس الرئاسي الوفاق لحكومة الوطني رئيس السراج ليبيا
Topic 6: اللي مش علي البلاد ليبيا بس توا فينا
Topic 7: ليبيا المتحدة أن الليبي المؤتمر الآن الأمم لنتفق
Topic 8: لنتفق الآن لنتفق_الآن type مدينة طرابلس إلى تشجيع
Topic 9: الله لله الملك مافي مابعده timeline الآن photos
Topic 10: أن إلى حكومة الوفاق مجلس السياسي الوطني المجلس
Topic 11: ﺍﻟﻠﻪ ﺍﻛﺒﺮ ﻣﻦ ﺑﺎﻟﻠﻪ داعش ﺇﻻ قوات ﻭﻻ
Topic 12: ال الله وا ين الل ون وسلم صل
Topic 13: أن ليبيا داعش إلى إن الليبيين الشعب الاتحاد
Topic 14: ليبيا المركزي مصرف دولار دينار الليبية السيولة المواطن
Topic 15: اللهم الله يا أن يارب العظيم لي خير
Topic 16: طرابلس النفط الوطنية المنطقة بنغازي أي المدني أن
Topic 17: ليبيا الشعب الجيش الليبي الي الغرب العالم النظام
Topic 18: علي بنغازي يا

## Finding matching topics
We know that only some topics will overlap between the two sets, and that the similar topics may be in a different order. We can easily measure the similarity between two topics by listing the top 100 words in each topic, and counting how many of those top words are in common between the two lists. If two topics have more than 50% of their top 100 words in common, we'll consider them to be a match.

In [34]:
def overlap(td1,td2,n=10):
    w1 = top_words(td1,n)
    w2 = top_words(td2,n)
    return(len(set(w1) & set(w2))/n)

match_pairs = []
n = tw1.shape[0]
for i in range(n):
    for j in range(n):
        o = overlap(tw1[i,:],tw2[j,:],100)
        if o > 0.5:
            match_pairs.append((i,j))
            
print(match_pairs)

[(0, 9), (2, 7), (2, 10), (3, 6), (5, 0), (7, 15), (9, 2), (12, 1), (13, 12), (15, 14), (16, 5), (17, 8)]


Out of our two sets of 20 topics, we have 12 matches. The numbers above mean that, for example, topic \#0 from the first model matches with \#9 from the second model, while topic \#2 from the first model could match with \#7 or \#10 from the second.
## Example posts
Each post has a probability -- a number between 0 and 1 -- measuring how likely it is to fit in with any topic. To better understand what is in a topic, we can find a post with a very high probability for that topic. The example below is for topic \#9 in the first model, which (as far as I can tell from Google Translate) contains terms related to "national", "government", "conference", and "accord."

In [37]:
# Show key words and example post for a given match pair
def topic_example(i,n=10000):
    print(long_posts.Contents[np.argmax(ldamat1[:,i])][:n])

i = 9
topic_example(i)

لنتفق الآن

https://www.facebook.com/Lets.Agree.LY/photos/a.1500669000237089.1073741828.1500579190246070/1504781203159202/?type=3

لنتفق الآن

موجز رقم 2

موجز لأبرز ما تداولته وسائل الإعلام اليوم الأثنين 21.12.2015 .

ومواصلة في رصد اخر اخبار ملف الحوار الليبي - اللبيبيفقد ذكر السيد " سعيد الختالي "عضو المؤتمر الوطني العام ن المؤتمر ومجلس النواب شكلا يوم السبت الماضي لجنة مشتركة للحوار بينهما، مكونة من 34 عضوا نصفهم من المؤتمر الوطني ونصفهم الآخر من مجلس النواب، وبين الختالي وبيّن الختالي أن للجنة المشتركة مهام عدة أولها تشكيل حكومة وفاق وطني. وعلى صعيدا اخر اصدر مجلس البحوث والدراسات الشرعية بدارالإفتاء بيانا عبر فيه عن دعمه للحوار الليبي الليبي المتمثل في إعلان المبادئ بتونس ، ودعا المجلس في بيانه كل من المؤتمر الوطني ومجلس النواب إلى الإسراع بتشكيل حكومة وفاق وطني تنهي الانقسام في السلطة ، مطالبا المجتمع الدولي بالتريث ودعم الوفاق الليبي الليبي. هذا وتجدر الإشارة ان السيد " عقيلة صالح " رئيس مجلس النواب قد طالب المجتمع الدولي بدعم الحوار الليبي الليبي الذي استهل بلقاء بينه وبين رئي